In [5]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250

cpu


In [6]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [3]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)

['\n', ' ', '!', '"', '&', "'", ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [7]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
# print(data[:100])

In [8]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

inputs:
tensor([[56, 55, 62, 62,  1, 51, 69, 62],
        [ 1, 59, 69,  1, 54, 51, 64, 57],
        [54,  1, 75, 65, 71,  1, 62, 59],
        [52, 68, 59, 57, 58, 70,  1, 70]])
targets:
tensor([[55, 62, 62,  1, 51, 69, 62, 55],
        [59, 69,  1, 54, 51, 64, 57, 55],
        [ 1, 75, 65, 71,  1, 62, 59, 61],
        [68, 59, 57, 58, 70,  1, 70, 58]])


In [9]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [10]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


,RkCEP"h3m461D]g.?OldrQGs!Dm&M.dl2]U:.UcVl'abLSg5eN'bg-W5?_YIM! Y:Lzk,wYxIkT5dceU"nC:.hwt&asi]39Y[MGhZm&wswf= L0q8RDg"h,p59KY56P5AgYJwT5k_CJcmFg4S8]E:.U:,LG_0zi1L0DBhafIJ&5ber9VfZ8i1JT:b9p3Qh.&'GpWBwefMtmY4ef=mr
[VuW! rsiHxBA="v8Bh6Z1"rl=e
,ey:H_'oruyhmGi'd;Db"M_uJlg40mZnBR1"R[3:J1Z[w;7
iYDioUf9C']q8,CD7vPsPB,xI﻿n!gW:.h.RlAf?"k1jtFjpTp12p hOW2CQO;5U:Ww:LN8TMdjy3p;,iO
y=d:AuP50
7[p;=ECeJ?-p;2CgAaIx,qCISS1Wp1.!GSp"I7F0Jin
J4dzG﻿! w;H,;myY[2M'3hC:df, 
Zq!HJbhq ksx7]qfl0A9Ax"9TubR4eU3D-AD1GJ DW]q-!3


In [11]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 4.823, val loss: 4.818
step: 250, train loss: 4.776, val loss: 4.768
step: 500, train loss: 4.689, val loss: 4.721
step: 750, train loss: 4.630, val loss: 4.645
4.520163059234619


In [12]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


JGWMnDbjyqIMtw﻿B_fMtlif9YG1xIL0?zkgw;GuUk"RIijbMhgTt_CcHxRMmdcZ=csiCcM g[-=! H7RVE0TH:V4w6!HtWwT][VsTpjHYqHfc_CrM.r.LGhm&9xJxS?qt?;c4ebqcLScS:;eabOJfQRlDjq-WvUcF FV' 4SrQGsgsp7cM0rhfKc pWIp-DL[58'W.bgVwygWyxt43]50Sg5k=-qk9;3Sp51R﻿9HY4;KztIeaaIe':.
boQ00bR-"3N-=3]_b8FAotlaNAQ"Wv[kS3JTOyB!jonnm&VbgFjy:qBaTHzrdsihh7jEnacq-"h?A]6=m8;"rdzl=
lP=﻿5_S?ZZGPJed
qw]jSU&CD9&V4k,gDB0ikgwei;EfK50"'PRe_&6nIrk9Sc[1sTQQIBi;cU5gsTKey"d_Q3iPRYqQdoP]PVS:x:TC:.&'PcZ bRjoJ7JU&6﻿1We
Z8,!8]U! br:"W
i'WyyVw =mO9Kc6g[uDo
